<a href="https://colab.research.google.com/github/phoenix-1-2/Neural-Machine-Translation/blob/main/Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Reading Text and Pairing french words with English Words
texts = open('/content/fra.txt').readlines()
input_seq = []
for text in texts:
  words = text.lower().split('\t')
  try:
    input_seq.append( [ words[0] , words[1] ] ) 
  except:
    continue
input_seq

[['go.', 'va !'],
 ['hi.', 'salut !'],
 ['hi.', 'salut.'],
 ['run!', 'cours\u202f!'],
 ['run!', 'courez\u202f!'],
 ['who?', 'qui ?'],
 ['wow!', 'ça alors\u202f!'],
 ['fire!', 'au feu !'],
 ['help!', "à l'aide\u202f!"],
 ['jump.', 'saute.'],
 ['stop!', 'ça suffit\u202f!'],
 ['stop!', 'stop\u202f!'],
 ['stop!', 'arrête-toi !'],
 ['wait!', 'attends !'],
 ['wait!', 'attendez !'],
 ['go on.', 'poursuis.'],
 ['go on.', 'continuez.'],
 ['go on.', 'poursuivez.'],
 ['hello!', 'bonjour !'],
 ['hello!', 'salut !'],
 ['i see.', 'je comprends.'],
 ['i try.', "j'essaye."],
 ['i won!', "j'ai gagné !"],
 ['i won!', "je l'ai emporté !"],
 ['i won.', 'j’ai gagné.'],
 ['oh no!', 'oh non !'],
 ['attack!', 'attaque !'],
 ['attack!', 'attaquez !'],
 ['cheers!', 'santé !'],
 ['cheers!', 'à votre santé !'],
 ['cheers!', 'merci !'],
 ['cheers!', 'tchin-tchin !'],
 ['get up.', 'lève-toi.'],
 ['go now.', 'va, maintenant.'],
 ['go now.', 'allez-y maintenant.'],
 ['go now.', 'vas-y maintenant.'],
 ['got it!', "j'a

In [14]:
import string

In [15]:
# Cleaning Pairs by removing puncitations
input_seq = [ [a.translate(str.maketrans('', '', string.punctuation)),b.translate(str.maketrans('', '', string.punctuation))  ] for a,b in input_seq ]

In [16]:
input_seq

[['go', 'va '],
 ['hi', 'salut '],
 ['hi', 'salut'],
 ['run', 'cours\u202f'],
 ['run', 'courez\u202f'],
 ['who', 'qui '],
 ['wow', 'ça alors\u202f'],
 ['fire', 'au feu '],
 ['help', 'à laide\u202f'],
 ['jump', 'saute'],
 ['stop', 'ça suffit\u202f'],
 ['stop', 'stop\u202f'],
 ['stop', 'arrêtetoi '],
 ['wait', 'attends '],
 ['wait', 'attendez '],
 ['go on', 'poursuis'],
 ['go on', 'continuez'],
 ['go on', 'poursuivez'],
 ['hello', 'bonjour '],
 ['hello', 'salut '],
 ['i see', 'je comprends'],
 ['i try', 'jessaye'],
 ['i won', 'jai gagné '],
 ['i won', 'je lai emporté '],
 ['i won', 'j’ai gagné'],
 ['oh no', 'oh non '],
 ['attack', 'attaque '],
 ['attack', 'attaquez '],
 ['cheers', 'santé '],
 ['cheers', 'à votre santé '],
 ['cheers', 'merci '],
 ['cheers', 'tchintchin '],
 ['get up', 'lèvetoi'],
 ['go now', 'va maintenant'],
 ['go now', 'allezy maintenant'],
 ['go now', 'vasy maintenant'],
 ['got it', 'jai pigé '],
 ['got it', 'compris '],
 ['got it', 'pigé\u202f'],
 ['got it', 'compris\

In [17]:
# Spiliting Testing and Training Data
import numpy as np
input_seq = np.array(input_seq)
from sklearn.model_selection import train_test_split
training_data,testing_data = train_test_split(input_seq)

In [18]:
# French and English Tokeniser
from keras.preprocessing.text import Tokenizer
french_tokenizer = Tokenizer()
english_tokenizer  = Tokenizer()

In [19]:
# Fiting our data in french and english tokenizer
french_tokenizer.fit_on_texts(training_data[:,0] )
english_tokenizer.fit_on_texts(training_data[:,1] )

In [20]:
# Getting Sequence from from training text
french_seq= french_tokenizer.texts_to_sequences(training_data[:,0])
english_seq= english_tokenizer.texts_to_sequences(training_data[:,1])

In [21]:
# Gettigng max length of sequence
max_french = max([ len(s) for s in french_seq ])
max_english = max([ len(s) for s in english_seq ])

In [22]:
# importing pad sequence
from keras.preprocessing.sequence import pad_sequences

In [23]:
# Padding French Seq to make all sequence of same size
french_seq=pad_sequences(french_seq ,maxlen=max_french )

In [24]:
# Padding English Seq to make all sequence of same size
english_seq=pad_sequences(english_seq ,maxlen=max_english )

In [25]:
english_seq.shape,french_seq.shape

((133506, 55), (133506, 44))

In [26]:
# # Taking a lot of RAM and crashing as Output data is huge 
# from keras.utils import to_categorical
# label = to_categorical(english_seq)

In [27]:
# Making Decoding Layer
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Bidirectional,RepeatVector
model = Sequential()
model.add(Embedding( len(french_tokenizer.word_index)+1 , 100, input_length=french_seq.shape[1]))
model.add(LSTM(1024))

In [28]:
# Adding RepeatVector Layer to make encoder ouput size same as decoder layer
model.add(RepeatVector(english_seq.shape[1]  ))

In [29]:
# Making Decoder layer
model.add(LSTM(512))
model.add(Dense(english_seq.shape[1], activation='softmax'))

In [30]:
# Compiling Model
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'] )

In [31]:
# Fitting our Frencg and English Training
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='accuracy', min_delta=0.01)
model.fit(french_seq,english_seq,epochs=20,batch_size=512, callbacks=[es] )

Epoch 1/20
261/261 [==============================] - 77s 295ms/step - loss: 93552.4688 - accuracy: 0.4304
Epoch 2/20
261/261 [==============================] - 80s 307ms/step - loss: 111550.1250 - accuracy: 0.4322


In [32]:
# Converting Testing Data into sequence and padding them
french_seq_test= french_tokenizer.texts_to_sequences(testing_data[:,0])
english_seq_test= english_tokenizer.texts_to_sequences(testing_data[:,1])
english_seq_test=pad_sequences(english_seq ,maxlen=max_english )
french_seq_test=pad_sequences(french_seq ,maxlen=max_french )

In [34]:
# Score of Testig Data
model.evaluate(french_seq_test,english_seq_test)

4173/4173 [==============================] - 54s 13ms/step - loss: 120760.0312 - accuracy: 0.4322


[120760.03125, 0.43217533826828003]